# Overview

This is for the second round of testing.  **Updated 25 July 2022**. The second round of testing will involve the files WriteToAirtable_test2.ipynb and 'secrets2.json' and will be focused on using an account sid from Airtable to (1) query data from Presto (probably for a subset of accounts) and (2) update one or more fields in Airtable. Again, we're just trying to see if things can be done at all. We might want to go ahead and scale to using Airflow at this stage.

In [1]:
import pandas as pd
import numpy as np
import os
import json 
import requests
from airtable import airtable #You need to do a pip install airtable.  Page at https://pypi.org/project/airtable/

In [2]:
#Read the secrets file
def read_secrets(fn) -> dict:
    filename = os.path.join(fn)
    try:
        with open(filename, mode='r') as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}

In [ ]:
#note the API base id is specific to the Airtable "base".  So, adjust the secrets and the ID to fit the specific Airtable base.
#the api key is an account-wide value so shouldn't change.
secret = read_secrets('secrets2.json')

In [4]:
#Start by downloading data from Airtable
#https://towardsdatascience.com/downloading-airtable-data-into-python-89e5c7107a24
#reading participant database - Jayson copy.  Will need to change table id in secrets.json to use the real one.

#Get a table in the Airtable base
#see https://github.com/josephbestjames/airtable.py

at = airtable.Airtable(secret['AIRTABLE_BASE_ID'], secret['AIRTABLE_API_KEY'])

In [63]:
#Note, only returns 1 page = 100 records per request.  So, we'll need to do pagination.

#Basically I want to write a recursion to make the next call until the current dict size is x<100, then exit.
#But, before that, I'll get two pages and merge?  Or, should I work with them separately?  Could do either, maybe I don't need to merge.

def get_all_records(pages,nums,offset=None,num_recs=100,count=0):
    count = count + 1
    at_dict = at.get(secret['AIRTABLE_TABLE_ID'],offset=offset)
    pages.append(at_dict)
    if 'offset' in at_dict:
        offset = at_dict['offset']
    else: #if there is no offset, that means that we hit the last page
        num_recs = len(at_dict['records'])
        nums.append(num_recs)
        print(count,num_recs,offset)
        return((pages,nums))
        exit
    num_recs = len(at_dict['records'])
    nums.append(num_recs)
    print(count,num_recs,offset) #this is for debugging.  Remove in production version.
    return(get_all_records(offset = offset,num_recs = num_recs, pages = pages,nums = nums,count=count))

In [ ]:
#For now, need to pass these blank lists.  Might be a better way, like tuples or something.
pages = []
nums = []
all_pages = get_all_records(pages=pages,nums=nums)

In [94]:
print(f'{sum(all_pages[1])} :Airtable records')
print(f'{len(all_pages[0])} :Number of dicts')

12546 :Airtable records
126 :Number of dicts


In [106]:
#If there are duplicate records, we won't have the same number of records as we do unique Record ID fields.
at_dict = all_pages[0][0]
type(at_dict)
at_dict['records']

[OrderedDict([('id', 'rec00O8V71VwSCFNI'),
              ('createdTime', '2022-07-25T20:31:22.000Z'),
              ('fields',
               OrderedDict([('Company Revenue', []),
                            ('Record ID', 'recOmiPjQ8ki0pxWJ'),
                            ('Voice Channels', ['']),
                            ('Segment Personas Features', []),
                            ('Name', 'Klaus Thorup'),
                            ('Job Title', 'Co-founder'),
                            ('Industry', []),
                            ('Responsibility level: Driving retention/repeat purchase (from Email)',
                             []),
                            ('Programmable Voice Channels', ['']),
                            ('Years Coding', '5+ years'),
                            ('Preferred Programming language (from Filter survey responses)',
                             []),
                            ('Data Errors', []),
                            ('Flex: CC Role D

In [ ]:
# for i in range(rec):
#     f = at_dict['records'][i]['fields']
#     id = at_dict['records'][i]['id']
#     n = at_dict['records'][i]['fields']['Email']
#     if n == 'ayushman.sinhaa@gmail.com':
#         print(i,id)

In [ ]:
#A field to update
#update_data = {"Email": "updated@viaAPI.com"}
#update_data

In [ ]:
#Update the email address for a given record based on the record id (generated by AirTable)
#Here, the id was obtained by matching the record id for name 'Nthabi Choma'
#at.update(secret['AIRTABLE_TABLE_ID'], id, update_data)